In [1]:
import init

import os
import pandas as pd

from Class import URL
from const import COMMON_CODE_URL
from common_code_controller import get_common_code

In [4]:
# 대분류는 상위 분류가 없으므로 제외

params = {
    'authKey':os.getenv('HRD_API_KEY'),
    'returnType':'XML',
    'outType':'1',
    'srchType':None,
    'srchOption1':None,
    'srchOption2':None
}
url = URL()
url.set_url(COMMON_CODE_URL)

# srchType


00 : 훈련지역 대분류 코드<br/>
01 : 훈련지역 중분류 코드<br/>
02 : KECO 대분류 코드<br/>
03 : KECO 중분류 코드<br/>
04 : KECO 소분류 코드<br/>
05 : NCS 대분류 코드<br/>
06 : NCS 중분류 코드<br/>
07 : NCS 소분류 코드<br/>
08 : NCS 세분류 코드<br/>
09 : 훈련종류 코드<br/>
10 : 훈련방법 코드<br/>
11 : 훈련기관 구분코드<br/>


# srchOption1

01 : 훈련지역 중분류 코드<br/>
03 : KECO 중분류 코드<br/>
04 : KECO 소분류 코드<br/>
06 : NCS 중분류 코드<br/>
07 : NCS 소분류 코드<br/>
08 : NCS 세분류 코드<br/><br/>

※사용예시)공통코드구분을 01(훈련지역 중분류)로 지정하고, 검색값에 11(서울) 입력시 서울의 훈련지역 중분류 코드들을 출력합니다.


# srchOption2

지원하는 공통코드 구분<br/>
04 : KECO 소분류 코드 [ 1자리 ex)1 ]<br/>
07 : NCS 소분류 코드 [ 2자리 ex)01 ]<br/>
08 : NCS 세분류 코드 [ 2,4자리 ex)01, 0101 ]<br/><br/>

※srchOption1이 있어야 사용 가능합니다.

In [3]:
# 훈련지역 중분류
region = pd.read_csv('../files/common_code/훈련지역_대분류_코드.csv')[['코드값','코드명']].values

params['srchType'] = '01'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i,j in region:
    params['srchOption1'] = str(i)
    url.set_parameter(params)
    
    middle = get_common_code(url)
    middle['시도_코드'] = i
    middle['시도'] = j
    
    df = pd.concat([df,middle])
df.columns = ['시군구_코드','시군구명','시도_코드','시도명']
df.to_csv('../files/common_code/훈련지역_중분류_코드_최종.csv',index=False)

In [4]:
# KECO 중분류
keco_main = pd.read_csv('../files/common_code/KECO_대분류_코드.csv',dtype={'코드값':str,'코드명':str})[['코드값','코드명']].values

params['srchType'] = '03'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i,j in keco_main:
    params['srchOption1'] = i
    url.set_parameter(params)
    
    middle = get_common_code(url)
    middle['대분류_코드'] = i
    middle['대분류_명'] = j
    
    df = pd.concat([df,middle])
df.columns = ['중분류_코드','중분류_명','대분류_코드','대분류_명']
df.to_csv('../files/common_code/KECO_중분류_코드_최종.csv',index=False)

In [5]:
# KECO 소분류
keco_middle = pd.read_csv('../files/common_code/KECO_중분류_코드_최종.csv',dtype={
    '중분류_코드':str,'중분류_명':str,'대분류_코드':str,'대분류_명':str
    })

params['srchType'] = '04'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i in keco_middle['중분류_코드'].values:
    middle_code = i
    params['srchOption1'] = i[:2]
    params['srchOption2'] = i[2]
    
    url.set_parameter(params)
    sub = get_common_code(url)
    sub['대분류_코드'] = i[:2]
    sub['중분류_코드'] = i
    
    df = pd.concat([df,sub])
df.columns = ['소분류_코드','소분류_명','대분류_코드','중분류_코드']
df.to_csv('../files/common_code/KECO_소분류_코드_최종.csv',index=False)

In [7]:
# NCS 중분류
ncs_main = pd.read_csv('../files/common_code/NCS_대분류_코드.csv',dtype={'코드값':str,'코드명':str})

params['srchType'] = '06'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i,j in ncs_main.values:
    params['srchOption1'] = i
    url.set_parameter(params)
    
    middle = get_common_code(url)
    middle['대분류_코드'] = i
    middle['대분류_명'] = j
    
    df = pd.concat([df,middle])
df.columns = ['중분류_코드','중분류_명','대분류_코드','대분류_명']
df.to_csv('../files/common_code/NCS_중분류_코드_최종.csv',index=False)

In [6]:
# NCS 소분류
ncs_middle = pd.read_csv('../files/common_code/NCS_중분류_코드_최종.csv',dtype={'중분류_코드':str,'중분류_명':str,'대분류_코드':str,'대분류_명':str})

params['srchType'] = '07'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i in ncs_middle['중분류_코드'].values:
    params['srchOption1'] = i[:2]
    params['srchOption2'] = i[2:]
    url.set_parameter(params)
    
    sub = get_common_code(url)
    sub['대분류_코드'] = i[:2]
    sub['중분류_코드'] = i
    
    df = pd.concat([df,sub])
df.columns = ['소분류_코드','소분류_명','대분류_코드','중분류_코드']
df.to_csv('../files/common_code/NCS_소분류_코드_최종.csv',index=False)

In [ ]:
# NCS 세분류
ncs_middle = pd.read_csv('../files/common_code/NCS_소분류_코드_최종.csv',dtype={'소분류_코드':str,'소분류_명':str,'대분류_코드':str,'중분류_코드':str})

params['srchType'] = '08'
params['srchOption1'] = None
params['srchOption2'] = None

df = pd.DataFrame({})
for i in ncs_middle['소분류_코드'].values:
    params['srchOption1'] = i[:2]
    params['srchOption2'] = i[2:]
    url.set_parameter(params)
    
    sub = get_common_code(url)
    sub['대분류_코드'] = i[:2]
    sub['중분류_코드'] = i[:4]
    sub['소분류_코드'] = i
    
    df = pd.concat([df,sub])
df.columns = ['소분류_코드','소분류_명','대분류_코드','중분류_코드','소분류_코드']
df.to_csv('../files/common_code/NCS_세분류_코드_최종.csv',index=False)